# Product Injury
<font size=4 color='blue'>Understand and Prep Data - Codes</font>   
***  

**Project Summary:**   
The Consumer Product Safety Commission operations a surveillance system (NEISS) to track injury data related to consumer products. The data is collected from a representative sample of emergency rooms in the United Status. 
This project will examine the data from 2013 through 2022 to explore trends in product injuries resulting in emergency room visits.

**Notebook Scope:**  
This notebook includes code to review and prep the lookup codes used in the NEISS data files. The data is pulled from each annual injury data file, which includes a lookup code worksheet. 

**Output:**  
An excel file containins a scrubbed lookup table that will be used for analysis of NEISS data.
***  

***
# Notebook Setup
***

In [1]:
# Import libraries
import pandas as pd

***  
# Load Data
***

In [2]:
# Read in the NEIS_FMT worksheet from each annual injury data Excel file
path = '../Data/'
files = ['neiss' + str(x) + '.xlsx' for x in range(2013, 2023)]
all_codes_df = pd.DataFrame()

for file in files:
    raw_df = pd.read_excel(path + file, sheet_name = 'NEISS_FMT')
    raw_df.insert(0, 'Year', file[-9:-5])
    all_codes_df = pd.concat([all_codes_df, raw_df])

In [3]:
# Reset index
all_codes_df.reset_index(drop=True, inplace=True)

***
# Preview Data
***

In [4]:
# Preview the dataframe
all_codes_df.head()

,Year,Format name,Starting value for format,Ending value for format,Format value label
0,2013,AGELTTWO,0,0,UNK
1,2013,AGELTTWO,2,120,2 YEARS AND OLDER
2,2013,AGELTTWO,201,201,1 MONTH
3,2013,AGELTTWO,202,202,2 MONTHS
4,2013,AGELTTWO,203,203,3 MONTHS


In [5]:
# Drop rows and columns that consist only of NaN data
all_codes_df.dropna(axis = 0, how = 'all', inplace=True)
all_codes_df.dropna(axis = 1, how = 'all', inplace=True)

In [6]:
# View shape of the dataframe
all_codes_df.shape

(12483, 5)

***  
# Variables
*** 

In [7]:
# Display variables (column headings) for the dataframe
all_codes_df.columns.values

array(['Year', 'Format name', 'Starting value for format',
       'Ending value for format', 'Format value label'], dtype=object)

***
**Variable Descriptions**  
-- Year: Indicates which annual injury data file the row was read from  
-- Format name: Code label  
-- Starting value for format: Starting value for the code label  
-- Ending value for format: Ending value for the code label    
-- Format value label: Description for code/label combination
***

In [8]:
# Review Year data
all_codes_df['Year'].unique()

array(['2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020',
       '2021', '2022'], dtype=object)

In [9]:
# Review Format name data
all_codes_df['Format name'].unique()

array(['AGELTTWO', 'ALC_DRUG', 'BDYPT', 'DIAG', 'DISP', 'FIRE', 'GENDER',
       'HISP', 'LOC', 'PROD', 'RACE'], dtype=object)

In [10]:
# Reivew Starting value for format data
all_codes_df['Starting value for format'].unique()

array(['               0', '               2', '             201', ...,
       '            9999', '             714', '            1552'],
      dtype=object)

In [11]:
# Review Ending value for format data
all_codes_df['Ending value for format'].unique()

array(['               0', '             120', '             201', ...,
       '            9999', '             714', '            1552'],
      dtype=object)

In [12]:
# Review Format value label data
all_codes_df['Format value label'].unique()

array(['UNK', '2 YEARS AND OLDER', '1 MONTH', ...,
       '714 - COMBINATION FIRE/SMOKE ALARM AND CARBON MONOXIDE DETECTORS',
       'NON-BINARY/OTHER', '1552 - CRIBS, NONPORTABLE OR NOT SPECIFIED'],
      dtype=object)

***
## Remove Irrelevant Data
***

In [13]:
# Review codes that encompass a range of values to see if we can simply to a single code/value pair instead of ranges
all_codes_df[all_codes_df['Starting value for format'] != all_codes_df['Ending value for format']]

,Year,Format name,Starting value for format,Ending value for format,Format value label
1,2013,AGELTTWO,2,120,2 YEARS AND OLDER
1249,2014,AGELTTWO,2,120,2 YEARS AND OLDER
2497,2015,AGELTTWO,2,120,2 YEARS AND OLDER
3745,2016,AGELTTWO,2,120,2 YEARS AND OLDER
4993,2017,AGELTTWO,2,120,2 YEARS AND OLDER
6241,2018,AGELTTWO,2,120,2 YEARS AND OLDER
7489,2019,AGELTTWO,2,120,2 YEARS AND OLDER
8737,2020,AGELTTWO,2,120,2 YEARS AND OLDER
9986,2021,AGELTTWO,2,120,2 YEARS AND OLDER
11235,2022,AGELTTWO,2,120,2 YEARS AND OLDER


In [14]:
# Our data analysis will focus on age in years only, so we can delete the Ending value for format and just use the starting value
all_codes_df.drop(columns='Ending value for format', inplace=True)

In [15]:
# Since ages over two do not need transalation, delete this row
rows_to_del = all_codes_df[all_codes_df['Format value label'] == '2 YEARS AND OLDER'].index.to_list()
all_codes_df.drop(rows_to_del, inplace=True)

In [16]:
# We are not going to consider alcohol use, drug use, or fire involvement in our analysis
rows_to_del = all_codes_df[all_codes_df['Format name'].isin(['ALC_DRUG', 'FIRE'])].index.to_list()
all_codes_df.drop(index=rows_to_del, inplace=True)

In [17]:
# Reset index
all_codes_df.reset_index(drop=True, inplace=True)

***
## Rename Variables for Clarity
***

In [18]:
# Rename columns
all_codes_df.columns = ['Year', 'Code', 'Value', 'Description']
all_codes_df.head()

,Year,Code,Value,Description
0,2013,AGELTTWO,0,UNK
1,2013,AGELTTWO,201,1 MONTH
2,2013,AGELTTWO,202,2 MONTHS
3,2013,AGELTTWO,203,3 MONTHS
4,2013,AGELTTWO,204,4 MONTHS


***
## Data Types and Formats
***

In [19]:
# Review column data types
all_codes_df.dtypes

Year           object
Code           object
Value          object
Description    object
dtype: object

In [20]:
# Convert Year to int
all_codes_df['Year'] = all_codes_df['Year'].astype(int)

In [21]:
# Strip spaces from Value data and check for non-numeric values
all_codes_df['Value'] = all_codes_df['Value'].str.strip()
for x in all_codes_df['Value'].unique():
    if x.isdigit() == False:
        print(all_codes_df[all_codes_df['Value'] == x])

       Year  Code Value     Description
96     2013  HISP     .  NA before 2019
1335   2014  HISP     .  NA before 2019
2574   2015  HISP     .  NA before 2019
3813   2016  HISP     .  NA before 2019
5052   2017  HISP     .  NA before 2019
6291   2018  HISP     .  NA before 2019
7530   2019  HISP     .  NA before 2019
8769   2020  HISP     .  NA before 2019
10010  2021  HISP     .  NA before 2019
11250  2022  HISP     .  NA before 2019


In [22]:
# Delete rows with "." as a Value. These rows are essentially comments
rows_to_del = all_codes_df[all_codes_df['Value'] == '.'].index.to_list()
all_codes_df.drop(index=rows_to_del, inplace=True)

In [23]:
# Convert Code to int
all_codes_df['Value'] = all_codes_df['Value'].astype(int)

In [24]:
# Review column data types
all_codes_df.dtypes

Year            int32
Code           object
Value           int32
Description    object
dtype: object

***
# Missing Data
***  

In [25]:
# Look for columns with null values
all_codes_df.columns[all_codes_df.isnull().any()]

Index(['Description'], dtype='object')

In [26]:
# Delete rows that do not contain a description
rows_to_del = all_codes_df[all_codes_df['Description'].isnull()].index
all_codes_df.drop(rows_to_del, axis='index', inplace=True)

***
# Duplicate Records
***

In [27]:
# We expect that we will have duplicate records from various years
all_codes_df.drop_duplicates(subset=['Code', 'Value', 'Description'], inplace=True)

In [28]:
# Look for duplicate code/value combinations that have different descriptions
all_codes_df[all_codes_df.duplicated(subset=['Code', 'Value'])]

,Year,Code,Value,Description


In [29]:
# Look for duplicate descriptions for different values by code
all_codes_df[all_codes_df.duplicated(subset=['Code', 'Description'])]

,Year,Code,Value,Description


In [30]:
# Reset index
all_codes_df.reset_index(drop=True, inplace=True)

***
# Data Review and Cleanup
***

## Year
***

In [31]:
# Review rows that are not from 2013
all_codes_df[all_codes_df['Year'] > 2013]

,Year,Code,Value,Description
1236,2020,PROD,714,714 - COMBINATION FIRE/SMOKE ALARM AND CARBON ...
1237,2021,GENDER,3,NON-BINARY/OTHER
1238,2021,PROD,1552,"1552 - CRIBS, NONPORTABLE OR NOT SPECIFIED"


In [32]:
# Check if these are new, or updated
display(all_codes_df[(all_codes_df['Code'] == 'PROD') & (all_codes_df['Value'].isin([714, 1552]))])
display(all_codes_df[(all_codes_df['Code'] == 'GENDER') & (all_codes_df['Value'] == 3)])

,Year,Code,Value,Description
1236,2020,PROD,714,714 - COMBINATION FIRE/SMOKE ALARM AND CARBON ...
1238,2021,PROD,1552,"1552 - CRIBS, NONPORTABLE OR NOT SPECIFIED"


,Year,Code,Value,Description
1237,2021,GENDER,3,NON-BINARY/OTHER


In [33]:
# Since there are no conflicts with the 3 newer codes, the Year column is not needed
all_codes_df.drop(columns='Year', inplace=True)

***
## Age, Gender
***

In [34]:
# Review age less than two data
all_codes_df[all_codes_df['Code'] == 'AGELTTWO']

,Code,Value,Description
0,AGELTTWO,0,UNK
1,AGELTTWO,201,1 MONTH
2,AGELTTWO,202,2 MONTHS
3,AGELTTWO,203,3 MONTHS
4,AGELTTWO,204,4 MONTHS
5,AGELTTWO,205,5 MONTHS
6,AGELTTWO,206,6 MONTHS
7,AGELTTWO,207,7 MONTHS
8,AGELTTWO,208,8 MONTHS
9,AGELTTWO,209,9 MONTHS


In [35]:
# Review gender data
all_codes_df[all_codes_df['Code'] == 'GENDER']

,Code,Value,Description
93,GENDER,0,UNKNOWN
94,GENDER,1,MALE
95,GENDER,2,FEMALE
1237,GENDER,3,NON-BINARY/OTHER


***
## Body Part
***

In [36]:
# Review body part data
all_codes_df[all_codes_df['Code'] == 'BDYPT']

,Code,Value,Description
24,BDYPT,0,0 - INTERNAL
25,BDYPT,30,30 - SHOULDER
26,BDYPT,31,31 - UPPER TRUNK
27,BDYPT,32,32 - ELBOW
28,BDYPT,33,33 - LOWER ARM
29,BDYPT,34,34 - WRIST
30,BDYPT,35,35 - KNEE
31,BDYPT,36,36 - LOWER LEG
32,BDYPT,37,37 - ANKLE
33,BDYPT,38,38 - PUBIC REGION


In [37]:
# Remove value from the description field
vals_to_update = all_codes_df.loc[all_codes_df['Code'] == 'BDYPT', 'Description']
for i, val in vals_to_update.iteritems():
    all_codes_df.loc[i, 'Description'] = " ".join(val[2:])

***
## Diagnosis
***

In [38]:
# Review diagnosis data
all_codes_df[all_codes_df['Code'] == 'DIAG']

,Code,Value,Description
54,DIAG,41,41 - INGESTION
55,DIAG,42,42 - ASPIRATION
56,DIAG,46,"46 - BURN, ELECTRICAL"
57,DIAG,47,"47 - BURN, NOT SPEC."
58,DIAG,48,"48 - BURN, SCALD"
59,DIAG,49,"49 - BURN, CHEMICAL"
60,DIAG,50,50 - AMPUTATION
61,DIAG,51,"51 - BURNS, THERMAL"
62,DIAG,52,52 - CONCUSSION
63,DIAG,53,"53 - CONTUSIONS, ABR."


In [39]:
# Remove value from the description field
vals_to_update = all_codes_df.loc[all_codes_df['Code'] == 'DIAG', 'Description']
for i, val in vals_to_update.iteritems():
    all_codes_df.loc[i, 'Description'] = val.split(' - ')[1]

***
## Disposition
***

In [40]:
# Review disposition data
all_codes_df[all_codes_df['Code'] == 'DIAG']

,Code,Value,Description
54,DIAG,41,INGESTION
55,DIAG,42,ASPIRATION
56,DIAG,46,"BURN, ELECTRICAL"
57,DIAG,47,"BURN, NOT SPEC."
58,DIAG,48,"BURN, SCALD"
59,DIAG,49,"BURN, CHEMICAL"
60,DIAG,50,AMPUTATION
61,DIAG,51,"BURNS, THERMAL"
62,DIAG,52,CONCUSSION
63,DIAG,53,"CONTUSIONS, ABR."


***
## Race, Hispanic
***

In [41]:
# Review race data
all_codes_df[all_codes_df['Code'] == 'RACE']

,Code,Value,Description
1229,RACE,0,N.S.
1230,RACE,1,WHITE
1231,RACE,2,BLACK/AFRICAN AMERICAN
1232,RACE,3,OTHER
1233,RACE,4,ASIAN
1234,RACE,5,AMERICAN INDIAN/ALASKA NATIVE
1235,RACE,6,NATIVE HAWAIIAN/PACIFIC ISLANDER


In [42]:
# Review Hispanic data
all_codes_df[all_codes_df['Code'] == 'HISP']

,Code,Value,Description
96,HISP,0,Unk/Not stated
97,HISP,1,Yes
98,HISP,2,No


***
## Location
***

In [43]:
# Review location data
all_codes_df[all_codes_df['Code'] == 'LOC']

,Code,Value,Description
99,LOC,0,UNK
100,LOC,1,HOME
101,LOC,2,FARM
102,LOC,3,APART.
103,LOC,4,STREET
104,LOC,5,PUBLIC
105,LOC,6,MOBILE
106,LOC,7,INDST.
107,LOC,8,SCHOOL
108,LOC,9,SPORTS


***
## Products
***

In [44]:
# Review products data
all_codes_df[all_codes_df['Code'] == 'PROD']

,Code,Value,Description
109,PROD,101,101 - WASHING MACHINES WITHOUT WRINGERS OR OTH...
110,PROD,102,102 - WRINGER WASHING MACHINES
111,PROD,103,103 - WASHING MACHINES WITH UNHEATED SPIN DRYERS
112,PROD,106,106 - ELECTRIC CLOTHES DRYERS WITHOUT WASHERS
113,PROD,107,107 - GAS CLOTHES DRYERS WITHOUT WASHERS
...,...,...,...
1226,PROD,9100,9100 - OUT OF SCOPE PRODUCT - RETAILER REPORT
1227,PROD,9200,9200 - DRYWALL CONTROL - NO COMPLAINT
1228,PROD,9999,9999 - UNCATEGORIZED PRODUCT
1236,PROD,714,714 - COMBINATION FIRE/SMOKE ALARM AND CARBON ...


In [45]:
# Remove value from the description field
vals_to_update = all_codes_df.loc[all_codes_df['Code'] == 'PROD', 'Description']
for i, val in vals_to_update.iteritems():
    all_codes_df.loc[i, 'Description'] = val.split(' - ')[1]

***
# Export to Excel
***

In [46]:
# Write final tabel to Excel
all_codes_df.to_excel('../Data/Standard Codes.xlsx', index=False)

***
**End**
***